In [12]:
import copy, numpy as np
import torch
import torch.nn as nn
import torchtext.data as data
from torch import optim
import torch.nn.functional as F
import re
import random
import unicodedata
import spacy
import string
from spacy.tokenizer import Tokenizer
from torchtext.data import Field
import en_core_web_sm
import csv
from torchtext.data import Dataset
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator
from torchtext.vocab import GloVe

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
np.random.seed(0)

In [35]:
torch.cuda.is_available()

False

Vectorizing data

In [13]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [14]:
def read_text():
    print("Reading training dataset from Stanford")
    lines = []
    with open('train.txt') as fp:
        for line in fp:
            line = re.sub(r"(your persona:.*\\n)", ' ', line)
            line = ' '.join(line.split())   
            question = re.findall(r"text:(.*)labels:", line)
            answer = re.findall(r"labels:(.*)(episode_done:)", line)
            if len(answer) == 0:
                answer = re.findall(r"labels:(.*)(question:)", line)
            if len(answer) and len(question):
                lines.append(question[0])
                lines.append(answer[0][0])
    
    return lines


In [15]:
def create_custom_tokenizer(nlp):
    custom_prefixes = [r'[0-9]+', r'\~', r'\–', r'\—', r'\$']
    custom_infixes = [r'[!&:,()]', r'\.', r'\-', r'\–', r'\—', r'\$']
    custom_suffixes = [r'\.', r'\–', r'\—', r'\$']
    default_prefixes = list(nlp.Defaults.prefixes) + custom_prefixes
    default_prefixes.remove(r'US\$')
    default_prefixes.remove(r'C\$')
    default_prefixes.remove(r'A\$')
    
    all_prefixes_re = spacy.util.compile_prefix_regex(tuple(default_prefixes))
    infix_re = spacy.util.compile_infix_regex(tuple(list(nlp.Defaults.infixes) + custom_infixes))
    suffix_re = spacy.util.compile_suffix_regex(tuple(list(nlp.Defaults.suffixes) + custom_suffixes))

    rules = dict(nlp.Defaults.tokenizer_exceptions)
    # remove "a." to "z." rules so "a." gets tokenized as a|.
    for c in range(ord("a"), ord("z") + 1):
        if f"{chr(c)}." in rules:
            rules.pop(f"{chr(c)}.")

    return Tokenizer(nlp.vocab, rules,
                     prefix_search=all_prefixes_re.search,
                     infix_finditer=infix_re.finditer, suffix_search=suffix_re.search,
                     token_match=None)

In [16]:
# We will use this special token to join the pre-tokenized data
JOIN_TOKEN = " "
nlp = en_core_web_sm.load()
nlp.tokenizer = create_custom_tokenizer(nlp)

def tokenize(text: string, tokenizer=nlp):
    tokens = [tok for tok in nlp.tokenizer(text) if not tok.text.isspace()]
    text_tokens = [tok.text for tok in tokens]
    return tokens, text_tokens


def tokenize_and_join(text: string, jointoken=JOIN_TOKEN):
    return jointoken.join(tokenize(text)[1])

In [17]:
lines = read_text()
print(len(lines))
print(tokenize_and_join(lines[0]))

Reading training dataset from Stanford
262876
hi , how are you doing ? i 'm getting ready to do some cheetah chasing to stay in shape .


In [18]:
def create_data(name, from_line, to_line):
    with open(name, mode='w') as csv_file:
        fieldnames = ['question', 'answer']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(from_line, to_line,2):
            writer.writerow({'question': lines[i], 'answer': lines[i+1]})


In [19]:
create_data('train.csv', 0, int(len(lines)*2/3))
create_data('valid.csv', int(len(lines)*2/3),len(lines))
create_data('test.csv', int(len(lines)/5), int(len(lines)/5*4))

In [20]:
class SDataset(Dataset):
    def __init__(self, sequences,**kwargs):
        super(SDataset, self).__init__(self,sequences, sequences,**kwargs) 
    def prepare_fields():
        TEXT = Field(sequential=True, tokenize=lambda s: str.split(s, sep=JOIN_TOKEN), lower=True)
        return [
            ('question',TEXT),
            ('answer',TEXT)   
        ]


In [24]:
config = { "train_batch_size": 5, "embedding_size": 100}
print(config)

data_fields = SDataset.prepare_fields()
fields = dict(tv_datafields)

train = SDataset(fields)
trn, vld, test = TabularDataset.splits(
        path="~/nlg",  # the root directory where the data lies
        train='train.csv', validation="valid.csv", test='test.csv',
        format='csv',
        skip_header=True,
        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
        fields=data_fields)

fields["question"].build_vocab(trn, vectors=GloVe(name='6B', dim=config["embedding_size"]))

vocab = fields["question"].vocab

train_iter = BucketIterator(train,
                            shuffle=True, sort=True, sort_within_batch=True,
                            sort_key=lambda x: len(x.question),
                            batch_size=config["train_batch_size"],
                            repeat=False,
                            device=device)
print(train_iter)
print(vocab.freqs.most_common(50))




{'train_batch_size': 5, 'embedding_size': 100}
[]


In [25]:
trn[0].__dict__.keys()
trn[0].question[:5]

['', 'hi', ',', 'how', 'are']

In [26]:
train_iter, val_iter = BucketIterator.splits(
 (trn, vld), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(64, 64),
 device=device, # if you want to use the GPU, specify the GPU number here
 sort_key=lambda x: len(x.question), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,
 repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

In [29]:
# create fields
fields = SDataset.prepare_fields()
print(fields)
fields = dict(fields)
train = SDataset(fields)
trn, vld, test = TabularDataset.splits(
    path="~/nlg",  # the root directory where the data lies
    train='train.csv', validation="valid.csv", test='test.csv',
    format='csv',
    skip_header=True,
    # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
    fields=data_fields)
fields["question"].build_vocab(trn, vectors=GloVe(name='6B', dim=config["embedding_size"]))


[('question', <torchtext.data.field.Field object at 0x7f689169c198>), ('answer', <torchtext.data.field.Field object at 0x7f689169c198>)]


In [30]:
MAX_LENGTH = 128

def filter_pair(p):
    return  len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def create_pairs(sentences):
    pairs = []
    for idx in range(0, len(sentences)-1, 2):
        pair = [sentences[idx], sentences[idx+1]]
        if filter_pair(pair):
            pairs.append(pair)
    return pairs



In [31]:
SOS_token = 0
EOS_token = 1

class Person:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
        

In [32]:
def prepare_data():
    data = read_text()
    pairs = create_pairs(data)
    print("data: ", data[:2])
    print("pair: ", pairs[0])
    input_data = Person("persona1")
    target_data = Person("persona2")
    for pair in pairs:
        input_data.addSentence(pair[0])
        target_data.addSentence(pair[1])
    print("Counted words:")
    print(input_data.name, input_data.n_words)
    print(target_data.name, target_data.n_words)
    return input_data, target_data, pairs

    

In [33]:
input_data, target_data, pairs = prepare_data()
test_input = "hi how are you"
for c in test_input.split():
    print("Word and Idx: ", c, input_data.word2index[c])

Reading training dataset from Stanford
data:  [" hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape . ", 'you must be very fast . hunting is one of my favorite hobbies . ']
pair:  [" hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape . ", 'you must be very fast . hunting is one of my favorite hobbies . ']
Counted words:
persona1 18081
persona2 18733
Word and Idx:  hi 3
Word and Idx:  how 5
Word and Idx:  are 6
Word and Idx:  you 7


In [34]:
def indexes_from_sentence(data, sentence):
    return [data.word2index[word] for word in sentence.split(' ')]

def tensor_from_sentence(data, sentence):
    indexes = indexes_from_sentence(data, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(input_data, pair[0])
    target_tensor = tensor_from_sentence(target_data, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

def tanh_prime(x):
    return  1 - np.tanh(x)**2

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input_text, hidden):
        embedded = self.embedding(input_text).view(1,1,-1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size) #Applies a linear transformation to the incoming data
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input_data, hidden):
        output = self.embedding(input_data).view(1,1,-1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length = MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_hidden[0, 0]
     
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder( decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach() #detach from history as input            
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length
            
        
        

In [ ]:
def train_iter(encoder, decoder, n_iters, print_every=1000, learning_rate = 0.01):
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate) #stochastic gradient descent 
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]
    
    criterion = nn.NLLLoss()
    print_loss_total = 0
    
    for it in range(1, n_iters+1):
        training_pair = training_pairs[it-1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        
        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('(%d %d%%) %.4f' % (it, it / n_iters * 100, print_loss_avg))
        
    

In [ ]:
hidden_size = 256
encoder = EncoderRNN(input_data.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, target_data.n_words).to(device)
#print("encoder: ", encoder)
#print("decoder: ", decoder)
train_iter(encoder, decoder, 50000, print_every=500)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(input_data, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
            
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoded_words = []
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(target_data.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
evaluateRandomly(encoder, decoder)